# 1. 라이브러리 가져오기

In [1]:
import pandas as pd
import re
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

# 2. 데이터 가져오기

In [2]:
ai_feeling = pd.read_csv('AI과정 느낌.csv', header=None, names=['feeling'])
ai_lesson = pd.read_csv('AI과정 교훈.csv', header=None, names=['lesson'])

app_feeling = pd.read_csv('앱과정 느낌.csv', header=None, names=['feeling'])
app_lesson = pd.read_csv('앱과정 교훈.csv', header=None, names=['lesson'])

print(ai_feeling)
print(ai_lesson)

print(app_feeling)
print(app_feeling)

FileNotFoundError: ignored

# 3. 데이터 전처리

column 이름이 같은 데이터 합침

In [ ]:
df_feeling = pd.concat([ai_feeling, app_feeling], ignore_index=True)
df_lesson = pd.concat([ai_lesson, app_lesson], ignore_index=True)

null 값 체크

In [ ]:
print(df_feeling.isnull().sum())
print(df_lesson.isnull().sum())

feeling    0
dtype: int64
lesson    0
dtype: int64


In [ ]:
feeling = []
feeling.extend(list(df_feeling.feeling.values))

lesson = []
lesson.extend(list(df_lesson.lesson.values))

print(feeling[:5])
print(lesson[:5])

['열심히 하자.', '오늘 실습 너무 쉬웠다. 역시 4조의 에이스는 나?', '오전에는 너무 어려웠는데 오후에 정리하면서 개념이 정리가 되어서 오늘 작성했던 코드가 다 이해가 되었다. 그래서 어려웠다고 생각했던 NLP에 한발짝 가까이 다가가게 된 것 같다.', '꾸역꾸역 따라가고 있다.^^', '실생활에 연계되거나 필요하다고 느꼈던 데이터에 직접 적용해보면 훨씬 집중된다']
['아래분 보고 배워야겠다.', '자신을 내보여라. 그러면 재능이 드러날 것이다.', '다각도에서 생각하는 능력을 기르자.', '영화 리뷰를 내가 입력해서 긍정/부정 여부를 확인해 봤는데 ‘사이다’, ‘적토마’ 등의 어휘가 갖는 의미를 인공지능이 인식을 못했다. co-lab으로 올려봐서 학습시켜 보면 더 정확히 인식할 것이다.', '개발자의 의도에 따라 또는 실수로 인해 결과가 편향될 수 있으므로 주의를 기울여서 조건을 설정해야한다']


In [ ]:
import re

# 정규표현식을 사용하여 특수문자를 제거하는 함수
def remove_special_characters(text):
    text = re.sub(r'[^\w\s]', '', text)  # 단어 문자와 공백 문자만 남김
    return text

# feeling 리스트의 각 문장에 대해 특수문자를 제거
cleaned_feeling = [remove_special_characters(sentence) for sentence in feeling]
cleaned_lesson = [remove_special_characters(sentence) for sentence in lesson]

# 결과 확인
print(cleaned_feeling)

['열심히 하자', '오늘 실습 너무 쉬웠다 역시 4조의 에이스는 나', '오전에는 너무 어려웠는데 오후에 정리하면서 개념이 정리가 되어서 오늘 작성했던 코드가 다 이해가 되었다 그래서 어려웠다고 생각했던 NLP에 한발짝 가까이 다가가게 된 것 같다', '꾸역꾸역 따라가고 있다', '실생활에 연계되거나 필요하다고 느꼈던 데이터에 직접 적용해보면 훨씬 집중된다', 'NLP 과정중 영화 평을 분석해 그사람의 감성 이 긍정인지 부정인지 판단하는 프로그램을 만들었다 LSTM으로 학습했고 결과는 81로 맞는다 프로그램을 만들면서 pandas의 dataframe을 사용하면 굉장히 편할것 같은데 그걸 안쓰고 만드느라 좀 이해하기 어려웠다 덕분에 리스트 컨프리헨션에 대해 공부하게 되었다 모르는거 물어보면 다 답해주는 짝궁이 있어서 굉장히 이해가 잘되었다', '왜 이렇게 할 게 많은지 모르겠다 수업 내용 복습하고 정리하는 것도 빡세고 개인적으로 해야하는게 다 기한이 촉박하다 매일 늦게까지 남아서 하고 가는데도 시간이 부족하다 그리고 2인 1조로 하는거 좋은 것 같다 같이 고민하니까 평소보다 탄력받아서 진행하게 된다 근데 신문고는 갑자기 무슨일이지 \xa0', '재즈 플리는 좋은데 인공지능 과정에서 듣기엔 기절하게 되는 포인트다 그래서 열심히 오프더레코드에 플리 올리고 있다 점심에 도우스피자 먹었는데 블루치즈앤베이컨이랑 버섯크림 짱맛 그리고 산책 너무 좋다', '깃 마스터 해야지 개발자 가자', '배우는게 좀 느리다', '차근차근 열심히 해야겠다', '여러가지를 동시에 하니까 과부하가 온 것 같다멍하다그렇지만 해야 한다 핵심 내용들은 꼭 짚고 넘어 가야지 나중에 힘들지 않을 것 같기 때문이다', '오늘의 실습으로 NLP 과정의 흐름이 어느정도 정리되는 느낌이었고 좀 더 개인적인 정리가 필요할 것 같다', 'NLP 들어와서 조금 멘붕이었는데 오늘 실습을 해보면서 쭉 정리할 수 있어서 너무 좋았다 근데 파이썬 문법에 익숙하지가 않아서 날코딩날코딩 할 일이 없다고 하더라도은 어렵다 

In [ ]:
feeling_tokenizer = Tokenizer()
feeling_tokenizer.fit_on_texts(cleaned_feeling)
feeling_vocab_size = len(feeling_tokenizer.word_index) + 1

In [ ]:
lesson_tokenizer = Tokenizer()
lesson_tokenizer.fit_on_texts(cleaned_lesson)
lesson_vocab_size = len(lesson_tokenizer.word_index) + 1

In [ ]:
sequences_feeling = list()

for line in cleaned_feeling:
    encoded = tokenizer.texts_to_sequences([line])[0] # 각  샘플에  대한  정수  인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences_feeling.append(sequence)

In [ ]:
sequences_lesson = list()

for line in cleaned_lesson:
    encoded = lesson_tokenizer.texts_to_sequences([line])[0] # 각  샘플에  대한  정수  인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences_lesson.append(sequence)

In [ ]:
sequences_feeling[:5]

[[9, 134], [6, 193], [6, 193, 3], [6, 193, 3, 580], [6, 193, 3, 580, 101]]

In [ ]:
feeling_max_len=max(len(l) for l in sequences_feeling) # 가장  긴  샘플의  길이  확인
print('샘플의  최대  길이  : {}'.format(feeling_max_len))

샘플의  최대  길이  : 75


In [ ]:
lesson_max_len=max(len(l) for l in sequences_lesson) # 가장  긴  샘플의  길이  확인
print('샘플의  최대  길이  : {}'.format(lesson_max_len))

샘플의  최대  길이  : 50


In [ ]:
sequences_feeling = pad_sequences(sequences_feeling,maxlen=feeling_max_len, padding = 'pre')
train_X_feeling = sequences_feeling[:,:-1]
train_y_feeling = sequences_feeling[:,-1]

In [ ]:
train_y_feeling = to_categorical(train_y_feeling, num_classes=feeling_vocab_size)
train_y_feeling

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
sequences_lesson = pad_sequences(sequences_lesson,maxlen=lesson_max_len, padding = 'pre')
train_X_lesson = sequences_lesson[:,:-1]
train_y_lesson = sequences_lesson[:,-1]

In [ ]:
train_y_lesson = to_categorical(train_y_lesson, num_classes=lesson_vocab_size)
train_y_lesson

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
feeling_model = Sequential()
# y데이터를  분리하였으므로  이제  X데이터의  길이는  기존  데이터의  길이  - 1
feeling_model.add(Embedding(feeling_vocab_size, 10, input_length=feeling_max_len-1)) # 10: 계산된 결과를 몇개까지 만들것인다
feeling_model.add(LSTM(128))
feeling_model.add(Dense(feeling_vocab_size, activation='softmax'))
feeling_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
feeling_model.fit(train_X_feeling, train_y_feeling, epochs=100, verbose=2)

Epoch 1/100
134/134 - 12s - loss: 7.8727 - accuracy: 0.0124 - 12s/epoch - 88ms/step
Epoch 2/100
134/134 - 9s - loss: 7.6622 - accuracy: 0.0126 - 9s/epoch - 70ms/step
Epoch 3/100
134/134 - 8s - loss: 7.4953 - accuracy: 0.0135 - 8s/epoch - 59ms/step
Epoch 4/100
134/134 - 9s - loss: 7.4479 - accuracy: 0.0117 - 9s/epoch - 64ms/step
Epoch 5/100
134/134 - 9s - loss: 7.3283 - accuracy: 0.0131 - 9s/epoch - 64ms/step
Epoch 6/100
134/134 - 8s - loss: 7.1993 - accuracy: 0.0152 - 8s/epoch - 58ms/step
Epoch 7/100
134/134 - 9s - loss: 7.0073 - accuracy: 0.0170 - 9s/epoch - 64ms/step
Epoch 8/100
134/134 - 9s - loss: 6.7502 - accuracy: 0.0187 - 9s/epoch - 64ms/step
Epoch 9/100
134/134 - 8s - loss: 6.4580 - accuracy: 0.0243 - 8s/epoch - 63ms/step
Epoch 10/100
134/134 - 8s - loss: 6.1562 - accuracy: 0.0254 - 8s/epoch - 60ms/step
Epoch 11/100
134/134 - 9s - loss: 5.8429 - accuracy: 0.0306 - 9s/epoch - 64ms/step
Epoch 12/100
134/134 - 9s - loss: 5.5477 - accuracy: 0.0394 - 9s/epoch - 65ms/step
Epoch 13/10

In [ ]:
lesson_model = Sequential()
# y데이터를  분리하였으므로  이제  X데이터의  길이는  기존  데이터의  길이  - 1
lesson_model.add(Embedding(lesson_vocab_size, 10, input_length=lesson_max_len-1)) # 10: 계산된 결과를 몇개까지 만들것인다
lesson_model.add(LSTM(128))
lesson_model.add(Dense(lesson_vocab_size, activation='softmax'))
lesson_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lesson_model.fit(train_X_lesson, train_y_lesson, epochs=100, verbose=2)

Epoch 1/100
48/48 - 4s - loss: 7.1224 - accuracy: 0.0099 - 4s/epoch - 78ms/step
Epoch 2/100
48/48 - 2s - loss: 7.0518 - accuracy: 0.0113 - 2s/epoch - 38ms/step
Epoch 3/100
48/48 - 3s - loss: 6.9678 - accuracy: 0.0079 - 3s/epoch - 55ms/step
Epoch 4/100
48/48 - 2s - loss: 6.8817 - accuracy: 0.0099 - 2s/epoch - 38ms/step
Epoch 5/100
48/48 - 2s - loss: 6.8361 - accuracy: 0.0139 - 2s/epoch - 38ms/step
Epoch 6/100
48/48 - 2s - loss: 6.7773 - accuracy: 0.0146 - 2s/epoch - 38ms/step
Epoch 7/100
48/48 - 2s - loss: 6.6690 - accuracy: 0.0132 - 2s/epoch - 38ms/step
Epoch 8/100
48/48 - 2s - loss: 6.5874 - accuracy: 0.0159 - 2s/epoch - 50ms/step
Epoch 9/100
48/48 - 3s - loss: 6.4286 - accuracy: 0.0132 - 3s/epoch - 61ms/step
Epoch 10/100
48/48 - 2s - loss: 6.2812 - accuracy: 0.0159 - 2s/epoch - 39ms/step
Epoch 11/100
48/48 - 2s - loss: 6.1233 - accuracy: 0.0165 - 2s/epoch - 38ms/step
Epoch 12/100
48/48 - 2s - loss: 5.9644 - accuracy: 0.0232 - 2s/epoch - 38ms/step
Epoch 13/100
48/48 - 2s - loss: 5.814

In [ ]:
def feeling_generation(model, t, current_word, n): # 모델, 토크나이저, 현재  단어, 반복핛  횟수
    init_word = current_word # 처음  들어온  단어도  마지막에  같이  출력하기위해  저장
    sentence = ''
    for _ in range(n): # n번  반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재  단어에  대핚  정수  인코딩
        encoded = pad_sequences([encoded], maxlen=74, padding='pre') # 데이터에  대한  패딩
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=-1)
        for word, index in t.word_index.items():
            if index == result: # 맊약  예측핚  단어와  인덱스와  동일한  단어가  있다면
                break # 해당  단어가  예측  단어이므로  break
        current_word = current_word + ' '  + word # 현재  단어  + ' ' + 예측  단어를  현재  단어로  변경
        sentence = sentence + ' ' + word # 예측  단어를  문장에  저장

    sentence = init_word + sentence
    return "느낌 : " + sentence

In [ ]:
def lesson_generation(model, t, current_word, n): # 모델, 토크나이저, 현재  단어, 반복핛  횟수
    init_word = current_word # 처음  들어온  단어도  마지막에  같이  출력하기위해  저장
    sentence = ''
    for _ in range(n): # n번  반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재  단어에  대핚  정수  인코딩
        encoded = pad_sequences([encoded], maxlen=49, padding='pre') # 데이터에  대한  패딩
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=-1)
        for word, index in t.word_index.items():
            if index == result: # 맊약  예측핚  단어와  인덱스와  동일한  단어가  있다면
                break # 해당  단어가  예측  단어이므로  break
        current_word = current_word + ' '  + word # 현재  단어  + ' ' + 예측  단어를  현재  단어로  변경
        sentence = sentence + ' ' + word # 예측  단어를  문장에  저장

    sentence = init_word + sentence
    return "교훈 : " + sentence

In [ ]:
print(feeling_generation(feeling_model, feeling_tokenizer, '오늘', 27))

느낌 : 오늘 하루가 지날 수록 점점 마음이 급해졌다 그래서 몇 가지 놓쳤는데 팀원이 도와주었다 어쨌든 오늘 개인 대화 기능 구현에 성공해서 마음이 놓였다 이제 팀원들을 볼 낯이 생겨서 다행이다


In [ ]:
print(lesson_generation(lesson_model, lesson_tokenizer, '오늘',17))

교훈 : 오늘 공부하자 새로운 게 계속 나오고 배운 거 활용하면서 디벨롭 해나가자 좋은 분들도 계시니 나만 더 잘하면 된다
